# 技術メモ: Phase 1 — 根本的な環境構築の見直し（Clean Image Strategy）

## 1. 結論

- Paperspace標準の「PyTorchランタイム」ではなく、**「Start from Scratch」**を選択する。
- 対策として、Docker Hubの公式安定版イメージ **`pytorch/pytorch:2.6.0-cuda12.4-cudnn9-devel`** を直接指定する。

**イメージ選定の根拠:**

- **PyTorch 2.6.0:** CUDA 12.4環境で動作する最新安定版
- **cuda12.4:** Paperspaceホスト側のNVIDIAドライババージョン（12040）に適合し、起動時の互換性エラーを回避
- **devel:** ComfyUIのカスタムノード追加時（C++/CUDA拡張）に必須となるコンパイラを内包

これにより、システム標準ライブラリとの競合（ImportError）とドライババージョン不一致を完全に排除する。

## 2. 推奨される環境構成

Notebook作成時の設定を以下に変更する。

| 項目 | 設定値 |
|------|--------|
| **Runtime** | Start from Scratch |
| **Advanced Options > Container** | `pytorch/pytorch:2.6.0-cuda12.4-cudnn9-devel` |
| **Advanced Options > Command** | 下記コマンド（Jupyter本体はシステム側にインストール。例外的に許容される構成） |

**Command（コピー用）:**

```
apt-get update && apt-get install -y git python3-venv && pip install jupyterlab --break-system-packages && PIP_DISABLE_PIP_VERSION_CHECK=1 jupyter lab --allow-root --ip=0.0.0.0 --no-browser --ServerApp.trust_xheaders=True --ServerApp.disable_check_xsrf=False --ServerApp.allow_remote_access=True --ServerApp.allow_origin='*' --ServerApp.allow_credentials=True
```

## 3. クリーンインストール手順（Jupyter Notebook用）

Jupyter環境のセルに以下のコードを貼り付けて**順に実行**する。  
ディレクトリ移動には状態を維持する `%cd` を使用し、コマンド実行には `!` を使用する。

In [1]:
# 1. ワークスペースの準備
%cd /notebooks
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI

/notebooks
fatal: destination path 'ComfyUI' already exists and is not an empty directory.
/notebooks/ComfyUI


In [2]:
# 2. 仮想環境（venv）の作成
# Ubuntu/Debian系イメージのため、事前に python3-venv がインストールされている必要がある
!python3 -m venv venv

In [3]:
# 3. 依存関係のインストール（仮想環境内のpipを直接指定）
# ホスト側のNVIDIAドライバ(12.4)に合わせたPyTorchをインストールする
!./venv/bin/pip install --upgrade pip
!./venv/bin/pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!./venv/bin/pip install -r requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu124


In [4]:
# 4. マネージャー導入
%cd custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ..

/notebooks/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-Manager' already exists and is not an empty directory.
/notebooks/ComfyUI


In [5]:
# 5. 起動とPublic URLの発行
# Paperspace固有の仕様を利用し、ポート6006で起動することでプロキシURLを生成する
import os
fqdn = os.environ.get('PAPERSPACE_FQDN', 'localhost')
print(f"\n🔗 ComfyUI Public URL: https://tensorboard-{fqdn}\n")

!./venv/bin/python main.py --listen 0.0.0.0 --port 6006


🔗 ComfyUI Public URL: https://tensorboard-nx4i4uawv2.clg07azjl.paperspacegradient.com

/notebooks/ComfyUI/venv/lib/python3.11/site-packages/requests/__init__.py:113: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (6.0.0.post1)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(
[START] Security scan
[ComfyUI-Manager] Using `uv` as Python module for pip operations.
Using Python 3.11.11 environment at: venv
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-23 15:47:49.293
** Platform: Linux
** Python version: 3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:17:24) [GCC 13.3.0]
** Python executable: /notebooks/ComfyUI/venv/bin/python
** ComfyUI Path: /notebooks/ComfyUI
** ComfyUI Base Folder Path: /notebooks/ComfyUI
** User directory: /notebooks/ComfyUI/user
** ComfyUI-Manager config path: /notebooks/ComfyUI/user/__manager/config.ini
** Log path: /notebooks/ComfyUI/user/comfyui.log
Using Pyt

: 

## 4. なぜこの方法が「推奨」なのか

| 観点 | 説明 |
|------|------|
| **システム保護** | `./venv/bin/pip` を使用することで、OS側のPythonパッケージ（Jupyter等）とComfyUI用のパッケージ（PyTorch等）が完全に分離される。 |
| **確実な実行** | パスを直接指定する方法は、Jupyterのマジックコマンドからでも確実に動作する。 |
| **ドライバ互換性** | Paperspaceホスト側のドライババージョン(12.4)に完全に適合し、起動エラーを回避できる。 |

## 5. 次のアクション

1. 現在エラーが出ているNotebookを**破棄（削除）**する。
2. 新規に**「Start from Scratch」**を選び、Containerに **`pytorch/pytorch:2.6.0-cuda12.4-cudnn9-devel`** を指定して作り直す。
3. 起動後、上記「3. クリーンインストール手順」を**再度実行**する。